In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pyvista as pv
from matplotlib.colors import ListedColormap

In [2]:
earthquake_data = pv.read('earthquake_data.vtk')
earthquake_data

PolyData (0x24d064ee400)
  N Cells:    1
  N Points:   31909
  N Strips:   0
  X Bounds:   -1.272e+07, -1.211e+07
  Y Bounds:   4.404e+06, 5.236e+06
  Z Bounds:   -9.990e+00, 6.628e+00
  N Arrays:   31

In [3]:
def get_mesh_by_year(year):
    lat_lng = []
    mags = []
    for i in range(len(earthquake_data.point_data['Lat'])):
        if earthquake_data.point_data['Year'][i] == year:
            lat_lng.append([earthquake_data.point_data['Long'][i], earthquake_data.point_data['Lat'][i], 
                        0])
            mags.append(earthquake_data.point_data['Mag'][i])
            
    my_mesh = pv.UnstructuredGrid()
    my_mesh.points = lat_lng
    my_mesh.point_data["Mag"] = mags
    return my_mesh

In [4]:
# Define hardcoded boundaries for Utah borders and quadrants
regional_verts = np.array([
    [[-111.55, 42.00, 0.0], [-111.55, 37.00, 0.0]],
    [[-114.05, 42.00, 0.0], [-109.05, 42.00, 0.0]]
])

utah_corners = np_points = np.array([
    [-114.05, 42.00, 0.0], 
    [-111.05, 42.00, 0.0],
    [-111.05, 41.00, 0.0],
    [-109.05, 41.00, 0.0],
    [-109.05, 37.00, 0.0],
    [-114.05, 37.00, 0.0]
])

In [5]:
# Generate sample mesh
my_mesh = get_mesh_by_year(2016)

In [6]:
# Define new colors for categorical mapping of estimated damage
c1 = np.array([0.99, 0.0, 0.0])
c2 = np.array([0.0, 0.99, 0.0])
c3 = np.array([0.0, 0.0, 0.99])
c4 = np.array([0.99, 0.99, 0.0])
c5 = np.array([0.99, 0.0, 0.99])
c6 = np.array([0.0, 0.99, 0.99])

# Map points to colors
mapping = np.linspace(my_mesh.point_data['Mag'].min(), my_mesh.point_data['Mag'].max(), 256)
new_colors = np.empty((256, 3))
new_colors[mapping >= 8.0] = c1
new_colors[mapping < 7.9] = c2
new_colors[mapping < 6.9] = c3
new_colors[mapping < 6.0] = c4
new_colors[mapping < 6.0] = c5
new_colors[mapping < 2.5] = c6

#Create new colormap object to be used in visualization
mag_classification_cmap = ListedColormap(new_colors)
lut = pv.LookupTable(cmap=mag_classification_cmap)

# Interpret colors into categories for visualization legend
legend_entries = []
legend_entries.append(['Usually not felt', c6])
legend_entries.append(['Minor damage', c5])
legend_entries.append(['Slight damage', c4])
legend_entries.append(['Lots of damage', c3])
legend_entries.append(['Serious damage', c2])
legend_entries.append(['Destroys entire communities', c1])

In [7]:
pl = pv.Plotter(notebook=False)

outline_mesh = pv.PolyData(utah_corners)
pl.add_mesh(outline_mesh, point_size=15)

poly_line = pv.MultipleLines(points=utah_corners)
pl.add_mesh(poly_line, line_width=2)

line = pv.Line(pointa=utah_corners[0], pointb=utah_corners[5])
pl.add_mesh(line, line_width=2, color='white')

pl.add_mesh(my_mesh, point_size=2)

ball = pv.Sphere(radius=0.05, theta_resolution=35, phi_resolution=35)
mag_glyphs = my_mesh.glyph(geom=ball, orient=False, scale="Mag")
pl.add_mesh(mag_glyphs, cmap=lut)


pl.add_legend(legend_entries)

pl.show(cpos='xy')